In [ ]:
import sys, re
import numpy as np
import tensorflow as tf

Let's define our neural network model

In [ ]:
def model_function(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 1])
    hidden = tf.layers.dense(inputs=input_layer, units=1)
    logits = tf.layers.dense(inputs=hidden, units=2)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Next, lets define the training data. Remeber, numpy arrays are required for Tensorflow (and we've imported numpy as 'np').

In [ ]:
    train_data   = np.asarray( [1, 2, 3, 4, 5, 6, 7, 8], dtype=np.float32 )
    train_labels = np.asarray( [0, 0, 0, 0, 1, 1, 1, 1], dtype=np.int32 )

The actual data input object...
<li>x: If we want to present more than one feature, we have to give a dict, hence the "x" (we could've named it anything we wanted)
<li>y: labels tensor
<li>batch size: how many features to present at once
<li>num_epochs: how many times to present the complete data (or None, for no limit)
<li>shuffle: to randomize the order of features or not

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)

If you need to remove the model later (if you change the model or if you want to start training from the beginning), just remove the directory.

In [ ]:
import shutil
shutil.rmtree("examplemodel")

Let's define the estimator object, by through which the model is used. Notice we also define a directory where the model will be saved during training. If we wish to run more iterations of training, the previous state of the model will be automatically loaded from that directory.

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")

Now that we have the estimator (often called a classifier, which it actually is in this case), we can call train on it. Lets do 100 training steps.

In [ ]:
estimator.train( input_fn=train_input_fn, steps=10000 )

Now we can present the estimator with an evaluation data set and see what the accuracy of the trained model is.

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x":train_data},
  y=train_labels,
  num_epochs=1,
  shuffle=False)

In [ ]:
eval_results = estimator.evaluate(input_fn=eval_input_fn)

Now the model is trained, lets use it. Type in a number below to find which class it belongs to...

In [ ]:
test_data = [6]
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
print(list(eval_results))

You could also say:

In [ ]:
test_data = [2]
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
res = next(eval_results)
print("The number "+str(test_data[0])+" is "+("greater than 4" if res["classes"]==1 else "less than 5"))

Finally, lets look at the model's internals

In [ ]:
print(str(estimator.get_variable_names()))
print("WEIGHTS 0"+str(estimator.get_variable_value("dense/kernel")))
print("BIAS 0"+str(estimator.get_variable_value("dense/bias")))
print("WEIGHTS 1"+str(estimator.get_variable_value("dense_1/kernel")))
print("BIAS 1"+str(estimator.get_variable_value("dense_1/bias")))